In [1]:
%matplotlib inline
%load_ext autoreload
%autoreload 2


In [2]:
from eden.converter.molecule import obabel
import networkx as nx
import pybel
import requests
import os.path
from itertools import tee
from numpy.random import randint
from numpy.random import uniform
from eden.graph import Vectorizer
from sklearn.linear_model import SGDClassifier
import datetime, time
from eden.util import random_bipartition_iter
from eden.model import ActiveLearningBinaryClassificationModel

from eden.util import configure_logging
import logging
configure_logging(logging.getLogger(),verbosity=2)

In [3]:
def make_iterable(filename, file_format):
    if file_format == 'sdf':
        with open(filename) as f:
            s = ''
            for line in f:
                if line.strip() != '$$$$':
                    s = s + line
                else:
                    return_value = s + line
                    s = ''
                    yield return_value
    elif file_format == 'smi':
        with open(filename) as f:
            for line in f:
                yield line

This is where the data sets are defined:

In [4]:
AID = 720577
#AID=2401
DATA_DIR = '/home/liconj/proj/thesis/EDeN/examples/model_comparison/data'
active_fname=DATA_DIR + '/AID%s_active.sdf'%AID
inactive_fname=DATA_DIR + '/AID%s_inactive.sdf'%AID

___
___


### Original model

Functions for training and testing the model

In [9]:
model_fname = DATA_DIR + '/AID%s.model'%AID
model_type = "default"
n_conf = 10
n_iter = 50
active_set_size = 5
n_active_learning_iterations = 0
threshold = 1
train_test_split = 0.8


pre_processor_parameters={'k':randint(1, 10,size=n_iter),
                          'threshold':randint(3, 10, size=n_iter),
                          'model_type':[model_type],
                          'n_conf':[n_conf]}

def pre_processor(data, model_type="3d", **kwargs):
    # model_type = kwargs.get('mode', 'default')
    if model_type == "default":
        iterable = obabel.obabel_to_eden(data, **kwargs)
    elif model_type == "3d":
        iterable = obabel.obabel_to_eden3d(data, **kwargs)
    return iterable



vectorizer = Vectorizer()
estimator = SGDClassifier(class_weight='auto', shuffle=True)

# Make predictive model
model = ActiveLearningBinaryClassificationModel(pre_processor,
                                                estimator=estimator,
                                                vectorizer=vectorizer,
                                                n_jobs=2,
                                                pre_processor_n_jobs=2,
                                                n_blocks = 10,
                                                fit_vectorizer=True)

In [10]:
########
# Create iterables from files
########

iterable_pos = make_iterable('AID720577_active.sdf', 'sdf')
iterable_neg = make_iterable('AID720577_inactive.sdf', 'sdf')
iterable_pos, iterable_pos_ = tee(iterable_pos)
iterable_neg, iterable_neg_ = tee(iterable_neg)

start = time.time()
print('# positives: %d  # negatives: %d (%.1f sec %s)'%(sum(1 for x in iterable_pos_), sum(1 for x in iterable_neg_), time.time() - start, str(datetime.timedelta(seconds=(time.time() - start)))))


iterable_pos, iterable_pos_ = tee(iterable_pos)
iterable_neg, iterable_neg_ = tee(iterable_neg)

# Split train/test
iterable_pos_train, iterable_pos_test = random_bipartition_iter(iterable_pos, relative_size=train_test_split)
iterable_neg_train, iterable_neg_test = random_bipartition_iter(iterable_neg, relative_size=train_test_split)

# positives: 80  # negatives: 146 (0.0 sec 0:00:00.033643)


In [11]:
%%time
# Optimize hyperparameters and fit model
# Since this model is fitted much more slowly, use a single vectorizer
#vectorizer_parameters={'complexity':[2,3,4],
#                       'discretization_size':randint(2, 3,size=n_iter),
#                       'discretization_dimension':randint(2, 3,size=n_iter)}

vectorizer_parameters={'complexity':[4,5,6], 'n':[2,3,4]}


estimator_parameters={'n_iter':randint(5, 100, size=n_iter),
                      'penalty':['l1','l2','elasticnet'],
                      'l1_ratio':uniform(0.1,0.9, size=n_iter),
                      'loss':['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'],
                      'power_t':uniform(0.1, size=n_iter),
                      'alpha': [10**x for x in range(-8,-2)],
                      'eta0': [10**x for x in range(-4,-1)],
                      'learning_rate': ["invscaling", "constant", "optimal"]}

model.optimize(iterable_pos_train, iterable_neg_train,
               model_name=model_fname,
               n_active_learning_iterations=0,
               size_positive=-1,
               size_negative=active_set_size,
               n_iter=n_iter, cv=3,
               pre_processor_parameters=pre_processor_parameters,
               vectorizer_parameters=vectorizer_parameters,
               estimator_parameters=estimator_parameters)



	Parameters range:

Pre_processor:
         k: [7 9 2 4 4 4 9 3 5 6 1 5 5 8 7 5 3 3 7 7 8 7 4 2 2 5 3 7 6 3 9 7 5 8 8 5 4
 5 4 1 2 7 8 1 6 2 1 6 4 6]
model_type: ['default']
    n_conf: [10]
 threshold: [3 7 4 4 5 5 7 8 3 8 4 9 5 4 3 4 5 6 4 4 7 6 7 6 7 3 8 7 6 4 9 9 8 4 3 6 5
 5 7 6 3 4 4 7 6 8 5 4 8 3]

Vectorizer:
complexity: [4, 5, 6]
         n: [2, 3, 4]

Estimator:
     alpha: [1e-08, 1e-07, 1e-06, 1e-05, 0.0001, 0.001]
      eta0: [0.0001, 0.001, 0.01]
  l1_ratio: [ 0.43327578  0.56052373  0.43737127  0.83020261  0.36424524  0.33742733
  0.6089759   0.62673306  0.43114672  0.15488701  0.15440253  0.5460247
  0.33777066  0.69585548  0.53076951  0.86638284  0.76012843  0.49787998
  0.35911858  0.44262159  0.52212307  0.44626761  0.32580556  0.17722919
  0.72248603  0.3375028   0.67675282  0.30550687  0.17972145  0.27601451
  0.42755236  0.6657247   0.35551186  0.67083203  0.5927725   0.87851374
  0.4619831   0.88762778  0.13533071  0.26189968  0.63529594  0.8628093
  0.62129438

In [12]:
%%time
# Estimate predictive performance
model.estimate( iterable_pos_test, iterable_neg_test )


Classifier:
SGDClassifier(alpha=0.001, average=False, class_weight='auto', epsilon=0.1,
       eta0=0.01, fit_intercept=True, l1_ratio=0.63500080223480915,
       learning_rate='optimal', loss='squared_hinge', n_iter=39, n_jobs=1,
       penalty='elasticnet', power_t=0.91193735932595033,
       random_state=None, shuffle=True, verbose=0, warm_start=False)

Data:
Instances: 46 ; Features: 1048577 with an avg of 895 features per instance

Predictive performace estimate:
             precision    recall  f1-score   support

         -1       0.76      0.87      0.81        30
          1       0.67      0.50      0.57        16

avg / total       0.73      0.74      0.73        46

APR: 0.536
ROC: 0.625
CPU times: user 222 ms, sys: 99.8 ms, total: 322 ms
Wall time: 1.28 s


(0.53630790456766064, 0.625)

_____________
_____________
### 3D model

In [27]:
model_fname = DATA_DIR + '/AID%s.model3d'%AID
model_type = "3d"
n_conf = 10
n_iter = 200
active_set_size = 5
n_active_learning_iterations = 0
train_test_split = 0.8


pre_processor_parameters={'k':randint(1, 10,size=n_iter),
                          'threshold':randint(3, 10, size=n_iter),
                          'model_type':[model_type],
                          'n_conf':[n_conf]}

def pre_processor(data, model_type="3d", **kwargs):
    # model_type = kwargs.get('mode', 'default')
    if model_type == "default":
        iterable = obabel.obabel_to_eden(data, **kwargs)
    elif model_type == "3d":
        iterable = obabel.obabel_to_eden3d(data, **kwargs)
    return iterable



vectorizer = Vectorizer()
estimator = SGDClassifier(class_weight='auto', shuffle=True)

# Make predictive model
model3d = ActiveLearningBinaryClassificationModel(pre_processor,
                                                  estimator=estimator,
                                                  vectorizer=vectorizer,
                                                  n_jobs = 1,
                                                  pre_processor_n_jobs = 1,
                                                  n_blocks = 10,
                                                  fit_vectorizer=True)

In [28]:
########
# Create iterables from files
########

iterable_pos = make_iterable('AID720577_active.sdf', 'sdf')
iterable_neg = make_iterable('AID720577_inactive.sdf', 'sdf')
iterable_pos, iterable_pos_ = tee(iterable_pos)
iterable_neg, iterable_neg_ = tee(iterable_neg)

start = time.time()
print('# positives: %d  # negatives: %d (%.1f sec %s)'%(sum(1 for x in iterable_pos_), sum(1 for x in iterable_neg_), time.time() - start, str(datetime.timedelta(seconds=(time.time() - start)))))


iterable_pos, iterable_pos_ = tee(iterable_pos)
iterable_neg, iterable_neg_ = tee(iterable_neg)

# Split train/test
iterable_pos_train, iterable_pos_test = random_bipartition_iter(iterable_pos, relative_size=train_test_split)
iterable_neg_train, iterable_neg_test = random_bipartition_iter(iterable_neg, relative_size=train_test_split)

# positives: 80  # negatives: 146 (0.2 sec 0:00:00.213707)


In [29]:
model3d.fit_vectorizer

True

In [30]:
#%%time
# Optimize hyperparameters and fit model

vectorizer_parameters={'complexity':[6], 'n':[2,3,4]}


estimator_parameters={'n_iter':randint(5, 100, size=n_iter),
                      'penalty':['l1','l2','elasticnet'],
                      'l1_ratio':uniform(0.1,0.9, size=n_iter),
                      'loss':['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'],
                      'power_t':uniform(0.1, size=n_iter),
                      'alpha': [10**x for x in range(-8,-2)],
                      'eta0': [10**x for x in range(-4,-1)],
                      'learning_rate': ["invscaling", "constant", "optimal"]}

model3d.optimize(iterable_pos_train, iterable_neg_train,
               model_name=model_fname,
               n_active_learning_iterations=0,
               size_positive=-1,
               size_negative=active_set_size,
               n_iter=n_iter, cv=3,
               pre_processor_parameters=pre_processor_parameters,
               vectorizer_parameters=vectorizer_parameters,
               estimator_parameters=estimator_parameters)



	Parameters range:

Pre_processor:
         k: [2 8 2 7 8 2 2 1 8 3 7 4 5 6 7 4 9 7 5 4 9 8 4 3 4 8 2 6 3 9 7 9 6 2 6 5 3
 7 7 7 8 1 2 8 5 4 9 4 6 6 5 7 8 2 4 5 9 2 9 5 7 7 5 5 6 4 6 7 1 7 7 9 5 4
 7 6 6 3 3 8 7 2 8 8 6 5 8 8 4 9 9 5 1 3 4 3 2 7 3 2 2 7 6 6 3 3 8 5 7 8 9
 2 7 5 4 5 8 9 9 2 2 6 3 1 9 1 9 8 7 8 2 7 8 7 4 4 3 5 5 3 4 2 1 4 5 6 5 3
 5 8 7 3 2 6 6 9 2 6 3 9 5 1 1 1 2 7 1 6 4 2 8 8 1 4 9 5 3 7 6 6 8 2 7 1 1
 4 6 7 8 6 7 1 5 4 2 6 3 4 6 6]
model_type: ['3d']
    n_conf: [10]
 threshold: [4 5 6 6 8 8 6 6 7 9 4 7 3 9 9 7 4 3 6 6 3 4 3 8 5 6 6 3 7 8 3 7 9 8 3 3 4
 5 6 7 5 6 9 7 3 3 6 3 9 6 7 7 8 3 8 8 7 3 4 6 7 5 6 7 5 9 4 8 8 8 5 5 8 9
 3 7 4 6 7 3 9 7 5 6 3 6 9 7 9 8 5 4 5 4 6 3 8 5 3 7 5 9 5 4 8 6 6 8 5 5 5
 7 8 6 9 3 5 5 8 9 8 3 7 8 8 7 5 9 7 7 7 9 4 8 8 3 3 5 3 3 4 3 5 6 3 3 9 9
 6 5 5 9 8 4 5 5 4 3 4 7 4 9 3 7 5 8 3 5 4 5 5 9 7 6 5 4 4 8 7 6 4 9 4 7 3
 7 5 6 4 3 9 7 8 9 3 7 3 7 7 3]

Vectorizer:
complexity: [6]
         n: [2, 3, 4]

Estimator:
     alpha: [1e-08, 1e-07,

KeyboardInterrupt: 

In [ ]:
%%time
# Estimate predictive performance
model3d.estimate( iterable_pos_test, iterable_neg_test )

In [ ]:
def test_obabel_model(fname, model_type = "default", model_fname=None):
    from eden.model import ActiveLearningBinaryClassificationModel

    model = ActiveLearningBinaryClassificationModel()
    model.load(model_fname)

    #create iterable from files
    from eden.converter.molecule import obabel
    if model_type == "default":
        iterable=obabel.obabel_to_eden(fname)
    elif model_type == "3d":
        iterable=obabel.obabel_to_eden3d(fname)

    predictions= model.decision_function( iterable )

    return predictions